In [10]:
from absl import app
from absl import flags
from absl import logging
import tensorflow.compat.v1 as tf

from open_spiel.python import policy
from open_spiel.python import rl_environment
from open_spiel.python.algorithms import exploitability
from open_spiel.python.algorithms import policy_gradient

from open_spiel.python.games.leviathan_game import _NUM_PLAYERS, Action, LevithanState

game = "python_leviathan"
num_players = _NUM_PLAYERS

# env_configs = {"players": num_players}
env_configs = {}
env = rl_environment.Environment(game, **env_configs)

In [11]:
from ray.tune.registry import register_env
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.env.wrappers.open_spiel import OpenSpielEnv as OpenSpielEnvWrapper

register_env("leviathan", lambda config: OpenSpielEnvWrapper(env))

In [12]:
from ray.tune.registry import get_trainable_cls

try:
    # Check if the environment is registered correctly
    env_cls = get_trainable_cls("leviathan")
    print("Environment 'leviathan' is registered correctly.")
except Exception as e:
    print(f"Error: {e}")


Error: Unknown trainable: leviathan


In [13]:
import ray
from ray.rllib.algorithms.ppo import PPOConfig

ray.shutdown()
ray.init(ignore_reinit_error=True)

config = PPOConfig()
config['env'] = "leviathan"
config = config.training(gamma=0.9, lr=0.01, kl_coeff=0.3, train_batch_size=128)
config = config.resources(num_gpus=0)
config = config.env_runners(num_env_runners=1)
# config["log_level"] = "WARN"

algo = config.build(env="leviathan")
algo.train()

2024-06-29 14:39:38,338	INFO worker.py:1761 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
